# EPL DATA ANALYSIS

## Introduction 

The purpose of this notebook is to analyze the impact of covid-19 to the performance of English Premier League (EPL) teams. The dataset contained match information from : 
* 2018/2019 season (1 season before covid-19 interruption)
* 2019/2020 season (season where covid interrupted the league)
* 2020/2021 season (1 season after covid-19 interruption)

Information regarding the date of covid-19 interruption : 
* last match before covid-19 interruption = Leicester vs Aston Villa 10 March 2020 (match ID 46889)
* first match before covid-19 interruption = Aston Villa vs Sheffield United 18 June 2020 (match ID 46875)

Analysis objectives : 
* How the absence of spectators had impacted performance of EPL teams 
    * Home/Away Possession
    * Home/Away Shots on Target
    * Home/Away Shots
    * Home/Away Goals Scored per Game
    * Home/Away Win Rate
    * Home/Away Shot on Target % = Shots on Target / Shots
    * Home/Away Quantity Conversion Rate = Goals / Shots 
    * Home/Away Quality Conversion Rate  = Goals / Shots on Target
    * Home/Away Win Rate = No. of win / No. of games (Home/Away)
    * Home/Away Points per Game (PpG) = Average points/game (Home / Away)
* Relation of Ball Possession % vs Goal Conversion Rate
* Compare the performance metrics between "big-6" and "non big-6" teams before and after covid-19. (note : Big-6 = "Manchester United", "Manchester City", "Liverpool", "Arsenal", "Chelsea", "Tottenham Hotspurs")

In [2]:
import pandas as pd
#Set float number just have two decimals
pd.set_option('display.float_format','{:.2f}' .format)

In [3]:
epl_data = pd.read_csv('epl_data_cleaned.csv', index_col=[0])
epl_data

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,home_shots_on_target,away_shots_on_target,home_shots,away_shots,home_points,away_points
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,4,1,12,10,3,0
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,6,10,15,12,0,3
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,1,4,6,13,0,3
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,6,4,8,13,3,0
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,2,5,15,15,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,11,3,21,8,3,0
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,3,3,12,10,3,0
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,7,5,14,17,3,0
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,4,4,14,9,0,3


## Calculate Key Stats 

Some of the key stats required for further analysis are : 
* Shot on Target % = Shots on Target / Shots
* Quantity Conversion Rate = Goals / Shots 
* Quality Conversion Rate  = Goals / Shots on Target

In [4]:
def calculate_key_stats(df, num, div,stats_name,percentage=True):
    if percentage == True:
        #home key stats
        df['home_'+stats_name] = (df['home_'+num]/df['home_'+div]) * 100
        #away key stats
        df['away_'+stats_name] = (df['away_'+num]/df['away_'+div]) * 100
    else:
        #home key stats
        df['home_'+stats_name] = df['home_'+num]/df['home_'+div]
        #away key stats
        df['away_'+stats_name] = df['away_'+num]/df['away_'+div]
    
    #handling 0/0 value (if any)
    df = df.fillna(0, axis=1)
    
    return df

In [5]:
#calculate shot on target %
calculate_key_stats(df=epl_data, num='shots_on_target',div='shots',stats_name='SoT%')

#calculate quantity conversion rate
calculate_key_stats(df=epl_data, num='score',div='shots',stats_name='quan_CR', percentage=False)

#calculate quality conversion rate
calculate_key_stats(df=epl_data, num='score',div='shots_on_target',stats_name='qual_CR', percentage=False)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_shots,away_shots,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,12,10,3,0,33.33,10.00,0.17,0.00,0.50,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,15,12,0,3,40.00,83.33,0.00,0.17,0.00,0.20
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,6,13,0,3,16.67,30.77,0.00,0.23,0.00,0.75
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,8,13,3,0,75.00,30.77,0.25,0.08,0.33,0.25
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,15,15,0,3,13.33,33.33,0.07,0.13,0.50,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,21,8,3,0,52.38,37.50,0.24,0.00,0.45,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,12,10,3,0,25.00,30.00,0.08,0.00,0.33,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,14,17,3,0,50.00,29.41,0.21,0.00,0.43,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,14,9,0,3,28.57,44.44,0.07,0.22,0.25,0.50


## Calculate Points

In [6]:
def get_points(df):
    #home win
    df.loc[df['home_score']>df['away_score'], 'home_points'] = 3
    df.loc[df['home_score']>df['away_score'], 'away_points'] = 0

    #away win
    df.loc[df['home_score']<df['away_score'], 'home_points'] = 0
    df.loc[df['home_score']<df['away_score'], 'away_points'] = 3
    
    #draw
    df.loc[df['home_score']==df['away_score'], 'home_points'] = 1
    df.loc[df['home_score']==df['away_score'], 'away_points'] = 1
    
    return df
        
#calculate points for each game
get_points(epl_data)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_shots,away_shots,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,12,10,3,0,33.33,10.00,0.17,0.00,0.50,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,15,12,0,3,40.00,83.33,0.00,0.17,0.00,0.20
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,6,13,0,3,16.67,30.77,0.00,0.23,0.00,0.75
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,8,13,3,0,75.00,30.77,0.25,0.08,0.33,0.25
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,15,15,0,3,13.33,33.33,0.07,0.13,0.50,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,21,8,3,0,52.38,37.50,0.24,0.00,0.45,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,12,10,3,0,25.00,30.00,0.08,0.00,0.33,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,14,17,3,0,50.00,29.41,0.21,0.00,0.43,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,14,9,0,3,28.57,44.44,0.07,0.22,0.25,0.50


## Calculate Win Rate

In [8]:
def get_win_rate(df):
    #home win
    df.loc[df['home_score']>df['away_score'], 'home_win_rate'] = 1
    df.loc[df['home_score']>df['away_score'], 'away_win_rate'] = 0

    #away win
    df.loc[df['home_score']<df['away_score'], 'home_win_rate'] = 0
    df.loc[df['home_score']<df['away_score'], 'away_win_rate'] = 1
    
    #draw
    df.loc[df['home_score']==df['away_score'], 'home_win_rate'] = 0
    df.loc[df['home_score']==df['away_score'], 'away_win_rate'] = 0
    
    return df

#calculate win rate
get_win_rate(epl_data)

,match_id,match_date,matchweek,home_team,away_team,season,home_score,away_score,home_possession,away_possession,...,home_points,away_points,home_SoT%,away_SoT%,home_quan_CR,away_quan_CR,home_qual_CR,away_qual_CR,home_win_rate,away_win_rate
0,38309,2018-08-11,Matchweek 1,AFC Bournemouth,Cardiff City,2018/2019,2,0,62.90,37.10,...,3,0,33.33,10.00,0.17,0.00,0.50,0.00,1.00,0.00
1,38310,2018-08-11,Matchweek 1,Fulham,Crystal Palace,2018/2019,0,2,66.30,33.70,...,0,3,40.00,83.33,0.00,0.17,0.00,0.20,0.00,1.00
2,38311,2018-08-11,Matchweek 1,Huddersfield Town,Chelsea,2018/2019,0,3,37.20,62.80,...,0,3,16.67,30.77,0.00,0.23,0.00,0.75,0.00,1.00
3,38313,2018-08-11,Matchweek 1,Manchester United,Leicester City,2018/2019,2,1,46.30,53.70,...,3,0,75.00,30.77,0.25,0.08,0.33,0.25,1.00,0.00
4,38314,2018-08-11,Matchweek 1,Newcastle United,Tottenham Hotspur,2018/2019,1,2,40.40,59.60,...,0,3,13.33,33.33,0.07,0.13,0.50,0.40,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,59272,2021-05-23,Matchweek 38,Manchester City,Everton,2020/2021,5,0,67.70,32.30,...,3,0,52.38,37.50,0.24,0.00,0.45,0.00,1.00,0.00
1136,59273,2021-05-23,Matchweek 38,Sheffield United,Burnley,2020/2021,1,0,43.00,57.00,...,3,0,25.00,30.00,0.08,0.00,0.33,0.00,1.00,0.00
1137,59274,2021-05-23,Matchweek 38,West Ham United,Southampton,2020/2021,3,0,36.90,63.10,...,3,0,50.00,29.41,0.21,0.00,0.43,0.00,1.00,0.00
1138,59275,2021-05-23,Matchweek 38,Wolverhampton Wanderers,Manchester United,2020/2021,1,2,57.20,42.80,...,0,3,28.57,44.44,0.07,0.22,0.25,0.50,0.00,1.00


## Change Datatype

In [ ]:
epl_data.info()

In [ ]:
string_cols = ['matchweek','home_team','away_team','season']
int_cols = ['home_score','away_score','home_shots_on_target','away_shots_on_target',
            'home_shots','away_shots','home_points','away_points']
float_cols = ['home_possession','away_possession']
datetime_cols = 'match_date'

#convert to corresponding datatype
epl_data[string_cols] = epl_data[string_cols].astype("string")
epl_data[int_cols] = epl_data[int_cols].astype("int")
epl_data[float_cols] = epl_data[float_cols].astype("float")

#convert object to datetime object
epl_data['match_date'] = pd.to_datetime(epl_data['match_date'], format='%Y-%m-%d')

epl_data.info()

In [ ]:
#label match before and after covid19
epl_data.loc[epl_data['match_date']<='2020-03-10','covid19'] = 'before'
epl_data.loc[epl_data['match_date']>'2020-03-10','covid19'] = 'after'

epl_data

## Data Analysis

### Filter EPL Teams

The analysis will only observe the teams which had participated in all 3 seasons.

In [ ]:
#get teams name for every season since 2018/2019 
teams_18_19 = set(epl_data.loc[epl_data['season']=='2018/2019']['home_team'].unique())
teams_19_20 = set(epl_data.loc[epl_data['season']=='2019/2020']['home_team'].unique())
teams_20_21 = set(epl_data.loc[epl_data['season']=='2020/2021']['home_team'].unique())

#filter the team that had participated in all 3 seasons
filtered_teams = list(teams_18_19.intersection(teams_19_20,teams_20_21))
filtered_teams

### Home / Away Ball Possession %

In [ ]:
#home possession before and after covid
h_pos = pd.DataFrame()
h_pos['home_possession_before'] = epl_data.loc[epl_data['covid19']=='before'].groupby('home_team')\
                                        ['home_possession'].mean()
h_pos['home_possession_after'] = epl_data.loc[epl_data['covid19']=='after'].groupby('home_team')\
                                        ['home_possession'].mean()
h_pos['home_possession_changes'] = h_pos['home_possession_after']-h_pos['home_possession_before']

#drop na values, which eliminates teams that did not play in both before and after covid19
#h_pos.dropna(inplace=True)

#define covid19 effect on home possession
h_pos.loc[h_pos['home_possession_changes']>0,'effect'] = 'positive'  
h_pos.loc[h_pos['home_possession_changes']<0,'effect'] = 'negative'  
h_pos.loc[h_pos['home_possession_changes']==0,'effect'] = 'not affected'  

h_pos.sort_values('home_possession_changes',ascending=True)

In [ ]:
h_pos.value_counts('effect')

In [ ]:
#away possession before and after covid
a_pos = pd.DataFrame()
a_pos['away_possession_before'] = epl_data.loc[epl_data['covid19']=='before'].groupby('away_team')\
                                        ['away_possession'].mean()
a_pos['away_possession_after'] = epl_data.loc[epl_data['covid19']=='after'].groupby('away_team')\
                                        ['away_possession'].mean()
a_pos['away_possession_changes'] = a_pos['away_possession_after']-a_pos['away_possession_before']

#drop na values, which eliminates teams that did not play in both before and after covid19
a_pos.dropna(inplace=True)

#define covid19 effect on away possession
a_pos.loc[a_pos['away_possession_changes']>0,'effect'] = 'positive'  
a_pos.loc[a_pos['away_possession_changes']<0,'effect'] = 'negative'  
a_pos.loc[a_pos['away_possession_changes']==0,'effect'] = 'not affected'  

a_pos.sort_values('away_possession_changes',ascending=True)

In [ ]:
a_pos.value_counts('effect')